In [25]:
import ray
import modules.network_extractor as net_extractor
import geopandas as geopd
import pandas as pd
from shapely import ops
from osmnx import settings
import pyproj
from pathlib import Path
import networkx as nx
import osmnx as ox

In [26]:
# Change for your own data base path
data_base_path = "/home/geolab/Desktop/Research/data"

# The extractor instance
extractor = net_extractor.NetworkExtractor()
extractor.DATA_BASE_PATH = data_base_path
 
# Custom OSMnx settings
settings.default_crs = "epsg:4326"

ray.shutdown()
ray.init()

2025-04-30 12:02:33,578	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.11.3
Ray version:,2.42.1
Dashboard:,http://127.0.0.1:8265


(download_network pid=492099) extracting public_transport for sarajevo
(download_network pid=492102) extracting drive for sarajevo
(download_network pid=492101) extracting bike for sarajevo
(download_network pid=492103) extracting walk for sarajevo
(download_network pid=492099) finish processing
(download_network pid=492099) Final number of edges: 704
(download_network pid=492099) Final number of nodes: 482
(download_network pid=492102) finish download
(download_network pid=492102) Raw number of edges: 58322
(download_network pid=492102) Raw number of nodes: 30944
(download_network pid=492102) start rebuilding graph
(download_network pid=492102) finish rebuilding graph
(download_network pid=492102) start graph simplify
(download_network pid=492102) finish graph simplify
(download_network pid=492101) finish download [repeated 2x across cluster]
(download_network pid=492103) Raw number of edges: 110954 [repeated 2x across cluster]
(download_network pid=492103) Raw number of nodes: 51324 

In [27]:
# Extract information from the GHS dataset
urban_centers: geopd.GeoDataFrame

ghs_dataset_path = f"{data_base_path}/GHS_URBAN_CENTERS/GHS_URBAN_SIMPLIFIED_fixed.gpkg"
urban_centers = geopd.read_file(
    ghs_dataset_path,
    layer='GHS_URBAN_SIMPLIFIED_fixed'    
)
urban_centers = urban_centers.rename(columns={
    "GC_POP_TOT_2025": "population",
    "GC_UCA_KM2_2025": "area",
    "GC_DEV_USR_2025": "continent",
    "GC_UCN_MAI_2025": "name",
    "GC_CNT_GAD_2025": "country"
})
transform = pyproj.Transformer.from_crs("ESRI:54009", "EPSG:4326", always_xy=True).transform

In [28]:
capital_cities = f"{data_base_path}/capital_cities.csv"
capital_df = pd.read_csv(capital_cities, delimiter=",", header=None)
countries = list(capital_df[0])
city_names = list(capital_df[1])


In [29]:
capitals = pd.DataFrame()
for i in range(len(countries)):
    capital_city = urban_centers.loc[
        (urban_centers["name"] == city_names[i]) &
        (urban_centers["country"] == countries[i])
    ]
    capitals = pd.concat([capitals, capital_city])

capitals = capitals.reset_index(drop=True)

In [30]:
list(capitals["name"])

['Kabul',
 'Tirana',
 'Algiers',
 'Luanda',
 'Buenos Aires',
 'Yerevan',
 'Sydney',
 'Vienna',
 'Baku',
 'Nassau',
 'Manama',
 'Dhaka',
 'Bridgetown',
 'Minsk',
 'Brussels',
 'Belize City',
 'Porto-Novo',
 'Sucre',
 'Sarajevo',
 'Gaborone',
 'Brasilia',
 'Bandar Seri Begawan',
 'Sofia',
 'Ouagadougou',
 'Gitega',
 'Phnom Penh',
 'Yaoundé',
 'Ottawa',
 'Bangui',
 "N'Djamena",
 'Santiago',
 'Beijing',
 'Bogota',
 'Moroni',
 'Kinshasa',
 'Brazzaville',
 'San José',
 'Yamoussoukro',
 'Zagreb',
 'Havana',
 'Prague',
 'Copenhagen',
 'Djibouti',
 'Santo Domingo',
 'Dili',
 'Quito',
 'Cairo',
 'San Salvador',
 'London',
 'Malabo',
 'Asmara',
 'Tallinn',
 'Manzini',
 'Addis Ababa',
 'Suva',
 'Helsinki',
 'Paris',
 'Libreville',
 'Serrekunda',
 'Tbilisi',
 'Berlin',
 'Accra',
 'Athens',
 'Guatemala City',
 'Coyah',
 'Bissau',
 'Georgetown',
 'Port-au-Prince',
 'Tegucigalpa',
 'Budapest',
 'Reykjavik',
 'New Delhi',
 'Jakarta',
 'Tehran',
 'Baghdad',
 'Dublin',
 'Jerusalem',
 'Rome',
 'Kingston',

In [31]:

for cap in capitals.iterrows():
    # the city name in lowercase and slug_case for creating the folder to store the graphs and shapefiles
    city_name = cap[1][1].replace(" ", "_").lower()
    #city_name = "buenos_aires"

    # The name to search the city in the GHS dataset. Capital case. Also used for the DEM.
    search_name = cap[1][1]
    #search_name = "Buenos Aires"

    # The country in which the city is located for searching the GHS dataset. Capital case.
    country = cap[1][3]
    #country = "Argentina"

    # extract info from GHS with search_name and country
    city_info = urban_centers.loc[
        (urban_centers["name"] == search_name) &
        (urban_centers["country"] == country)
    ]

    if len(city_info) == 0:
        print(f"City {search_name}, {country} not found")

    else:
        geom = city_info["geometry"].values[0]
        geom = ops.transform(transform, geom)

    o_b = "{"
    c_b = "}"
    backslash = f"\\"
    specials = "áéíóú'șăŏã"
    replaces = "aeiou_saoa"

    display_name = city_name
    modified = False

    if "[" in display_name or "]" in display_name:
        modified  = True

    for i in range(len(specials)):
        if specials[i] in display_name:
            modified  = True
        display_name = display_name.replace(specials[i], replaces[i])
        display_name = display_name.replace("[", "")
        display_name = display_name.replace("]", "")

    ee_str = f"{o_b}'geometry': ee.Geometry.BBox{geom.bounds}, 'name': '{display_name}' {c_b},"

    # if modified:
    #     print(ee_str)
    

/tmp/ipykernel_470632/765246397.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_name = cap[1][1].replace(" ", "_").lower()
/tmp/ipykernel_470632/765246397.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  search_name = cap[1][1]
/tmp/ipykernel_470632/765246397.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  country = cap[1][3]


In [32]:
capitals = capitals.loc[capitals["name"] == "Sarajevo"]
capitals

,ID_UC_G0,name,GC_UCN_LIS_2025,country,GC_CNT_UNN_2025,area,population,GC_DEV_WIG_2025,continent,GC_PLS_SCR_2025,GC_UCB_YOB_2025,GC_UCB_YOD_2025,GC_UCM_CAP,geometry
18,426,Sarajevo,Sarajevo,Bosnia and Herzegovina,Bosnia and Herzegovina,61,235739.1356,Upper Middle,Europe,High,1975,2030,1,"MULTIPOLYGON (((1503001 5219001, 1506000 52190..."


In [33]:
errors = []
for cap in capitals.iterrows():
    # the city name in lowercase and slug_case for creating the folder to store the graphs and shapefiles
    city_name = cap[1][1].replace(" ", "_").lower()
    #city_name = "buenos_aires"

    # The name to search the city in the GHS dataset. Capital case. Also used for the DEM.
    search_name = cap[1][1]
    #search_name = "Buenos Aires"

    # The country in which the city is located for searching the GHS dataset. Capital case.
    country = cap[1][3]
    #country = "Argentina"

    o_b = "{"
    c_b = "}"
    backslash = f"\\"
    specials = "áéíóú'șăŏã"
    replaces = "aeiou_saoa"

    display_name = city_name
    modified = False

    if "[" in display_name or "]" in display_name:
        modified  = True

    for i in range(len(specials)):
        if specials[i] in display_name:
            modified  = True
        display_name = display_name.replace(specials[i], replaces[i])
        display_name = display_name.replace("[", "")
        display_name = display_name.replace("]", "")

    city_name = display_name

    try:
        # extract info from GHS with search_name and country
        city_info = urban_centers.loc[
            (urban_centers["name"] == search_name) &
            (urban_centers["country"] == country)
        ]

        if len(city_info) == 0:
            print(f"City {search_name}, {country} not found")

        else:
            geom = city_info["geometry"].values[0]
            geom = ops.transform(transform, geom)

        print(geom.bounds)

        # the geometry from which to extract the network. Is given by the GHS dataset.
        geometry = geom

        # Variables for the elimination of duplicate pedestrian/driving streets
        dist_threshold = 20
        slope_threshold = 15

        # Assessment = False, so duplicate pedestrian/driving streets will be eliminated.
        assess = False

        # Create graph and shapefile folders if they do not exist
        Path(f"{data_base_path}/{city_name}/graph").mkdir(parents=True, exist_ok=True) # graphml folder
        Path(f"{data_base_path}/{city_name}/shp").mkdir(parents=True, exist_ok=True) # shapefiles folder

        # Process the 4 networks in parallel using the paralellized download_network function.
        g_promises = []

        # the pedestrian network
        g_promises.append(extractor.download_network.remote(
            extractor,
            "walk", 
            geometry, 
            city_name, 
            assessment=assess, 
            dist_threshold=dist_threshold, 
            slope_threshold=slope_threshold,
            add_elevation=True,
        ))

        # the cycling network
        g_promises.append(extractor.download_network.remote(
            extractor,
            "bike", 
            geometry, 
            city_name, 
            assessment=assess, 
            add_elevation=True,
        ))

        # the driving network
        g_promises.append(extractor.download_network.remote(
            extractor,
            "drive", 
            geometry, 
            city_name, 
            assessment=assess, 
            add_elevation=True,
        ))

        # the public transport network
        g_promises.append(extractor.download_network.remote(
            extractor,
            "public_transport", 
            geometry, 
            city_name, 
            assessment=assess, 
            add_elevation=False,
        ))

        [g_walk, g_bike, g_drive, g_public] = ray.get(g_promises)

        # Save graphs
        extractor.save_as_graph(g_walk, f'{city_name}/graph/walk_{city_name}')
        extractor.save_as_graph(g_bike, f'{city_name}/graph/bike_{city_name}')
        extractor.save_as_graph(g_drive, f'{city_name}/graph/drive_{city_name}')
        extractor.save_as_graph(g_public, f'{city_name}/graph/public_{city_name}')

        # Save shapefiles
        extractor.save_as_shp(g_walk, f'{city_name}/shp/walk_{city_name}')
        extractor.save_as_shp(g_bike, f'{city_name}/shp/bike_{city_name}')
        extractor.save_as_shp(g_drive, f'{city_name}/shp/drive_{city_name}')
        extractor.save_as_shp(g_public, f'{city_name}/shp/public_{city_name}')
    except Exception as ex:
        errors.append(city_name)
        print(ex)

/tmp/ipykernel_470632/2370258519.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_name = cap[1][1].replace(" ", "_").lower()
/tmp/ipykernel_470632/2370258519.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  search_name = cap[1][1]
/tmp/ipykernel_470632/2370258519.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  country = cap[1][3]


(18.290715321625935, 43.80402003552376, 18.44633646029667, 43.895543123178676)
u
v
key
geometry
length
name
highway
bearing
foot
footway
speed_kph
travel_time
grade
grade_abs
access
sidewalk


/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:288: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nodes.to_file(f"{self.DATA_BASE_PATH}/{path}_nodes.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(
/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:290: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'travel_time' to 'travel_tim'
  ogr_write(


u
v
key
geometry
length
name
highway
lanes
speed_kph
travel_time
bearing
grade
grade_abs
cycleway
access
bicycle


/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:288: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nodes.to_file(f"{self.DATA_BASE_PATH}/{path}_nodes.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(
/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:290: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'travel_time' to 'travel_tim'
  ogr_write(


u
v
key
geometry
length
oneway
lanes
name
highway
maxspeed
bearing
speed_kph
travel_time
grade
grade_abs
ref
junction
width


/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:288: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nodes.to_file(f"{self.DATA_BASE_PATH}/{path}_nodes.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'street_count' to 'street_cou'
  ogr_write(
/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:290: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'travel_time' to 'travel_tim'
  ogr_write(


u
v
key
geometry
description
from
to
operator
network
type
route


/home/geolab/Desktop/Research/notebooks/modules/network_extractor.py:290: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  edges.to_file(f"{self.DATA_BASE_PATH}/{path}_edges.shp", encoding='utf-8')
/home/geolab/Desktop/Research/.venv/lib/python3.11/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'description' to 'descriptio'
  ogr_write(


In [8]:
errors

['kabul',
 'tirana',
 'algiers',
 'luanda',
 'buenos_aires',
 'yerevan',
 'sydney',
 'vienna',
 'baku',
 'nassau',
 'manama',
 'dhaka',
 'bridgetown',
 'minsk',
 'brussels',
 'belize_city',
 'porto-novo',
 'sucre',
 'sarajevo',
 'gaborone']

2025-04-24 18:20:08,728	ERROR worker.py:422 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): multiprocessing.pool.RemoteTraceback: 
  File "rasterio/_base.pyx", line 310, in rasterio._base.DatasetBase.__init__
  File "rasterio/_base.pyx", line 221, in rasterio._base.open_dataset
  File "rasterio/_err.pyx", line 359, in rasterio._err.exc_wrap_pointer
rasterio._err.CPLE_OpenFailedError: /home/geolab/Desktop/Research/data/DEM/Brasilia_DEM.tif: No such file or directory

During handling of the above exception, another exception occurred:

ray::download_network() (pid=399502, ip=10.78.3.89)
  File "/home/geolab/.pyenv/versions/3.11.3/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/home/geolab/.pyenv/versions/3.11.3/lib/python3.11/multiprocessing/pool.py", line 51, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [42]:
# for typing help
g_walk: nx.MultiDiGraph | nx.MultiGraph = g_walk

to_remove = []
for node in g_walk.nodes():
    for node_to in g_walk[node]:
        if len(list(g_walk[node][node_to].keys())) > 1:
            min_parallel = 99999999
            min_parallel_key = -1
            for parallel_edge_key in g_walk[node][node_to].keys():
                data = g_walk.get_edge_data(node, node_to, parallel_edge_key, default=None)
                if data["length"] > min_parallel:
                    min_parallel = data["length"]
                    min_parallel_key = parallel_edge_key
                
            for parallel_edge_key in g_walk[node][node_to].keys():
                if parallel_edge_key != min_parallel_key:
                    to_remove.append([node, node_to, parallel_edge_key])
            
    
    